In [11]:
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load in neccessary Data.
LA = pd.read_csv('LinkdeIn_Abouts.csv' ,encoding_errors='ignore')
LP = pd.read_csv('LinkdeIn_Publications.csv' ,encoding_errors='ignore')
CEC = pd.read_csv('profiles_final_all.csv')
GS = pd.read_csv('merge_google_asc.csv')
ASC = pd.read_csv('asc_consolidated_data.csv')

In [3]:
#Rename columns for LinkedIn Publications so column names are in common with CEC and Google Scholar files.
LP = LP.rename(columns={'Descriptions': 'descriptions','Title':'titles','Date':'date'})
LP

,Unnamed: 0,Name,titles,descriptions,date
0,0,Ahmed Bin Zaman,Guiding Protein Conformation Sampling with Con...,NaN,22-Mar-22
1,1,Ahmed Bin Zaman,Adaptive Stochastic Optimization to Improve Pr...,NaN,9-Dec-21
2,2,Ahmed Bin Zaman,Protein Decoy Generation via Adaptive Stochast...,NaN,19-Dec-20
3,3,Ahmed Bin Zaman,Reducing Ensembles of Protein Tertiary Structu...,NaN,9-May-20
4,4,Ahmed Bin Zaman,Decoy Ensemble Reduction in Template-free Prot...,NaN,1-Sep-19
...,...,...,...,...,...
369,42,Yue WANG,A robust and energy efficient cooperative spec...,NaN,1-Feb-09
370,43,Yue WANG,??????????????????????????????????????,NaN,1-Jul-08
371,44,Yue WANG,??????????????????????????,NaN,1-Jun-08
372,45,Yue WANG,????????????????????????????????????,NaN,1-Mar-07


In [4]:
#Rename columns for LinkedIn About sections so column names are in common with CEC and Google Scholar files.
LA = LA.rename(columns={'About': 'descriptions'})
LA

,Unnamed: 0,Name,descriptions
0,119,Abolfazl Safikhani,NaN
1,63,Aditya Johri,I'm Professor of Information Sciences & Tec...
2,16,Ahmed Bin Zaman,NaN
3,20,Alexander Brodsky,Alex Brodsky is a Professor in the departme...
4,78,Alexander Levis,NaN
...,...,...,...
169,28,Yue Cheng,"Distributed systems, storage systems, cloud..."
170,153,Yue Wang,I am currently a Research Assistant Professor ...
171,162,Zhisheng Yan,My research interests broadly lie in the broad...
172,173,Ziwei Zhu,Assistant Professor at the Department of Compu...


In [5]:
#Rename author column so it is the same name as the CEC name column.
GS = GS.rename(columns={'authors': 'Name'})
GS

,titles,Name,date,source,descriptions,citations
0,joint structural break detection parameter est...,abolfazl safikhani,1/2/2022,joint structural break detection parameter est...,assuming stationarity unrealistic many time se...,50.0
1,spatiotemporal modeling yellow taxi demands ne...,abolfazl safikhani,7/1/2020,spatiotemporal modeling yellow taxi demands ne...,spatiotemporal variation demand transportation...,37.0
2,investigation range anxiety safety buffer batt...,abolfazl safikhani,6/13/2018,investigation range anxiety safety buffer batt...,drivers tend range anxiety compared driving tr...,34.0
3,cyclelength prediction actuated trafficsignal ...,abolfazl safikhani,3/1/2018,cyclelength prediction actuated trafficsignal ...,urban transportation systems traffic signal ma...,32.0
4,predicting shortterm uber demand new york city...,abolfazl safikhani,5/1/2019,predicting shortterm uber demand new york city...,demand ehailing services growing rapidly espec...,17.0
...,...,...,...,...,...,...
14570,proceedings workshop natural language processi...,ziyu yao,2021/8,proceedings workshop natural language processi...,proliferation programmingrelated platforms git...,NaN
14571,advancing natural language interfaces data col...,ziyu yao,2021,advancing natural language interfaces data col...,natural language provides universal efficient ...,NaN
14572,iec towards interesteliciting neural conversat...,ziyu yao,2019,iec towards interesteliciting neural conversat...,conversational agents need sufficient social s...,NaN
14573,cashing,NaN,NaN,cashing,NaN,NaN


In [6]:
#take necessary columns from CEC and rename biography column so it is in common with the other files.
CEC = CEC[['Name', 'Biography']].copy()
CEC = CEC.rename(columns={'Biography': 'descriptions'})
CEC

,Name,descriptions
0,Abolfazl Safikhani,Abolfazl Safikhani is currently an assistant ...
1,Aditya Johri,Aditya Johri studies the use of information a...
2,Ahmed Bin Zaman,"“If they ever tell my story, let them say fri..."
3,Alexander Brodsky,Alex Brodsky joined George Mason University i...
4,Alexander Levis,Alexander H. Levis is University Professor of ...
...,...,...
237,Zelalem Eshete,Zelalem Eshete has taught at various colleges...
238,Zhisheng Yan,Zhisheng Yan is assistant professor in the De...
239,Ziwei Zhu,Ziwei Zhu is broadly interested in data minin...
240,Ziyu Yao,Ziyu Yao works on natural language processing...


In [235]:
#Combine Subjects, Keywords, and Abstract columns and rename column names.
ASC = ASC.replace({np.nan: 'None'})
ASC['descriptions'] = ASC['Subjects']+ ';' + ASC['Keywords']+ ';' +ASC['Abstract']
ASC = ASC[['Article Title','Author', 'descriptions','Publication Date']].copy() 
ASC = ASC.rename(columns={'Article Title': 'titles','Author':'Name','Publication Date':'date'})

In [236]:
ASC

,titles,Name,descriptions,date
0,Rescuing Prometheus: Four Monumental Projects ...,Aditya Johri,HISTORY of technological innovations; NONFICTI...,16-Oct
1,Autonomous Visual Perception for Unmanned Surf...,Aditya Johri,AUTONOMOUS vehicles; COMPUTER vision; PATTERN ...,19-May
2,Enhanced energy storage performance and therma...,Aditya Johri,HEAT storage; RELAXOR ferroelectrics; THERMAL ...,21-Jun
3,High energy storage performances of Bi1−xSmxFe...,Aditya Johri,LEAD-free ceramics; ENERGY storage; BISMUTH co...,19-Jul
4,High-Performance Nonequilibrium InSb PIN Infra...,Aditya Johri,PIN photodiodes; PHOTODETECTORS; CURRENT densi...,19-Mar
...,...,...,...,...
740,Modeling Human-in-the-Loop Security Analysis a...,Yue Wang,MATHEMATICAL models of decision making; WORKFL...,14-Jan
741,Reasoning with advanced policy rules and its a...,Yue Wang,ACCESS control; COMPUTER security; INFORMATION...,2004
742,Removing Permissions in the Flexible Authoriza...,Yue Wang,COMPUTER security; SECURITY systems; DATA prot...,3-Sep
743,Securing the ZigBee Protocol in the Smart Grid.,Yue Wang,COMPUTER network protocols; CYBERTERRORISM; CO...,12-Apr


In [237]:
#Concat data into one dataframe
data_concat = pd.concat([CEC,GS,ASC,LP,LA],ignore_index = True,sort=False)

In [239]:
#Replace empty cells or cells with messy data from scrapping with NAN
data_concat = data_concat.replace(r'^\s*$', np.nan, regex=True)
data_concat = data_concat.fillna('NNN/AAA')
data_concat = data_concat.apply(lambda x: x.astype(str).str.lower())
data_concat

,Name,descriptions,titles,date,source,citations,Unnamed: 0
0,abolfazl safikhani,abolfazl safikhani is currently an assistant ...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
1,aditya johri,aditya johri studies the use of information a...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
2,ahmed bin zaman,"“if they ever tell my story, let them say fri...",nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
3,alexander brodsky,alex brodsky joined george mason university i...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
4,alexander levis,alexander h. levis is university professor of ...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
...,...,...,...,...,...,...,...
16105,yue cheng,"distributed systems, storage systems, cloud...",nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,28.0
16106,yue wang,i am currently a research assistant professor ...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,153.0
16107,zhisheng yan,my research interests broadly lie in the broad...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,162.0
16108,ziwei zhu,assistant professor at the department of compu...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,173.0


In [240]:
#Remove whitesapce, -, and " from desciptions
data_concat['descriptions'] = data_concat['descriptions'].str.replace('\W', ' ', regex=True)# Removing special characters
data_concat['descriptions'] = data_concat['descriptions'].str.replace('-', ' ')
data_concat['descriptions'] = data_concat['descriptions'].str.replace('"', '')
data_concat

,Name,descriptions,titles,date,source,citations,Unnamed: 0
0,abolfazl safikhani,abolfazl safikhani is currently an assistant ...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
1,aditya johri,aditya johri studies the use of information a...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
2,ahmed bin zaman,if they ever tell my story let them say fri...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
3,alexander brodsky,alex brodsky joined george mason university i...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
4,alexander levis,alexander h levis is university professor of ...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
...,...,...,...,...,...,...,...
16105,yue cheng,distributed systems storage systems cloud...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,28.0
16106,yue wang,i am currently a research assistant professor ...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,153.0
16107,zhisheng yan,my research interests broadly lie in the broad...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,162.0
16108,ziwei zhu,assistant professor at the department of compu...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,173.0


In [241]:
#Add additional stop words to list
new_stopwords = ["A", "from", "In", "This", "We","-"] # removing Stop Words
stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(new_stopwords)

In [242]:
#Remove stop words from titles and descriptions
stop = stopwords.words('english') #Removing Stopwords

df2 = data_concat.copy()


df2["titles"] = df2["titles"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stpwrd)]))
df2['descriptions'] = df2['descriptions'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stpwrd)]))

df2 

,Name,descriptions,titles,date,source,citations,Unnamed: 0
0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
1,aditya johri,aditya johri studies use information communica...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
2,ahmed bin zaman,ever tell story let say friends called zoo bad...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
3,alexander brodsky,alex brodsky joined george mason university 19...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
4,alexander levis,alexander h levis university professor electri...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa
...,...,...,...,...,...,...,...
16105,yue cheng,distributed systems storage systems cloud comp...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,28.0
16106,yue wang,currently research assistant professor electri...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,153.0
16107,zhisheng yan,research interests broadly lie broad areas mob...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,162.0
16108,ziwei zhu,assistant professor department computer scienc...,nnn/aaa,nnn/aaa,nnn/aaa,nnn/aaa,173.0


In [243]:
#Removing Special Characters
df2['descriptions'] = df2['descriptions'].map(lambda x: re.sub(r'\W+', ' ', x)) 
df2['titles'] = df2['titles'].map(lambda x: re.sub(r'\W+', ' ', x))
df2['Name'] = df2['Name'].map(lambda x: re.sub(r'\W+', ' ', x))
df2 = df2.replace({'nnn/aaa': np.nan})
df2 = df2.replace({'nnn aaa': np.nan})
df2

,Name,descriptions,titles,date,source,citations,Unnamed: 0
0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,NaN,NaN,NaN,NaN,NaN
1,aditya johri,aditya johri studies use information communica...,NaN,NaN,NaN,NaN,NaN
2,ahmed bin zaman,ever tell story let say friends called zoo bad...,NaN,NaN,NaN,NaN,NaN
3,alexander brodsky,alex brodsky joined george mason university 19...,NaN,NaN,NaN,NaN,NaN
4,alexander levis,alexander h levis university professor electri...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
16105,yue cheng,distributed systems storage systems cloud comp...,NaN,NaN,NaN,NaN,28.0
16106,yue wang,currently research assistant professor electri...,NaN,NaN,NaN,NaN,153.0
16107,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,162.0
16108,ziwei zhu,assistant professor department computer scienc...,NaN,NaN,NaN,NaN,173.0


In [244]:
df2 = df2.fillna(value=np.nan)
df2

,Name,descriptions,titles,date,source,citations,Unnamed: 0
0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,NaN,NaN,NaN,NaN,NaN
1,aditya johri,aditya johri studies use information communica...,NaN,NaN,NaN,NaN,NaN
2,ahmed bin zaman,ever tell story let say friends called zoo bad...,NaN,NaN,NaN,NaN,NaN
3,alexander brodsky,alex brodsky joined george mason university 19...,NaN,NaN,NaN,NaN,NaN
4,alexander levis,alexander h levis university professor electri...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
16105,yue cheng,distributed systems storage systems cloud comp...,NaN,NaN,NaN,NaN,28.0
16106,yue wang,currently research assistant professor electri...,NaN,NaN,NaN,NaN,153.0
16107,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,162.0
16108,ziwei zhu,assistant professor department computer scienc...,NaN,NaN,NaN,NaN,173.0


In [245]:
#Remove few rows with many question marks
df2 = df2.applymap(lambda x: np.nan if x == '?' else x)
df2

,Name,descriptions,titles,date,source,citations,Unnamed: 0
0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,NaN,NaN,NaN,NaN,NaN
1,aditya johri,aditya johri studies use information communica...,NaN,NaN,NaN,NaN,NaN
2,ahmed bin zaman,ever tell story let say friends called zoo bad...,NaN,NaN,NaN,NaN,NaN
3,alexander brodsky,alex brodsky joined george mason university 19...,NaN,NaN,NaN,NaN,NaN
4,alexander levis,alexander h levis university professor electri...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
16105,yue cheng,distributed systems storage systems cloud comp...,NaN,NaN,NaN,NaN,28.0
16106,yue wang,currently research assistant professor electri...,NaN,NaN,NaN,NaN,153.0
16107,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN,162.0
16108,ziwei zhu,assistant professor department computer scienc...,NaN,NaN,NaN,NaN,173.0


In [246]:
#Remove unncessary columns
df2 = df2.drop(columns=['Unnamed: 0'])
df2

,Name,descriptions,titles,date,source,citations
0,abolfazl safikhani,abolfazl safikhani currently assistant profess...,NaN,NaN,NaN,NaN
1,aditya johri,aditya johri studies use information communica...,NaN,NaN,NaN,NaN
2,ahmed bin zaman,ever tell story let say friends called zoo bad...,NaN,NaN,NaN,NaN
3,alexander brodsky,alex brodsky joined george mason university 19...,NaN,NaN,NaN,NaN
4,alexander levis,alexander h levis university professor electri...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
16105,yue cheng,distributed systems storage systems cloud comp...,NaN,NaN,NaN,NaN
16106,yue wang,currently research assistant professor electri...,NaN,NaN,NaN,NaN
16107,zhisheng yan,research interests broadly lie broad areas mob...,NaN,NaN,NaN,NaN
16108,ziwei zhu,assistant professor department computer scienc...,NaN,NaN,NaN,NaN


In [13]:
#Adjust name difference from dataframe and CEC names
CEC = pd.read_csv('profiles_final_all.csv')
CEC = CEC.replace({np.nan: 'NNN/AAA'})
CEC = CEC.apply(lambda x: x.astype(str).str.lower())
CEC['Name'] = CEC['Name'].map(lambda x: re.sub(r'\W+', ' ', x))
CEC = CEC.replace({'nnn/aaa': 'N/A'})
CEC.to_csv('loadable_profiles_final_all.csv')
CEC

,ID,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link
0,171.0,abolfazl safikhani,assistant professor,N/A,abolfazl safikhani is currently an assistant ...,N/A,"phd, statistics, michigan state university\n",https://cec.gmu.edu/profiles/asafikha,asafikha@gmu.edu,https://sites.google.com/site/abolfazlsafikhan...
1,98.0,aditya johri,professor,703-993-5397,aditya johri studies the use of information a...,"personalized learning,","phd, learning technologies, stanford universi...",https://cec.gmu.edu/profiles/johri,johri@gmu.edu,http://mason.gmu.edu/~johri/\n
2,27.0,ahmed bin zaman,assistant professor\ncomputer science\ncollege...,N/A,"“if they ever tell my story, let them say fri...",N/A,"phd, computer science, george mason university...",https://cec.gmu.edu/node/6886,mailto;azaman6@gmu.edu,N/A
3,31.0,alexander brodsky,professor\ncomputer science\ncollege of engine...,703-993-1529,alex brodsky joined george mason university i...,"databases,","phd, computer science, the hebrew university ...",https://cec.gmu.edu/profiles/brodsky,brodsky@gmu.edu,http://cs.gmu.edu/~brodsky/\n
4,113.0,alexander levis,"professor emeritus, electrical and computer en...",703-993-1619,alexander h. levis is university professor of ...,"command and control, multi-formalism modeling,...","scd, mechanical engineering, mit\nme, mechani...",https://cec.gmu.edu/profiles/alevis,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...
237,54.0,zelalem eshete,assistant professor,N/A,zelalem eshete has taught at various colleges...,N/A,"phd, mechanical engineering, university of mar...",https://cec.gmu.edu/node/7986,zeshete@gmu.edu,N/A
238,223.0,zhisheng yan,assistant professor,N/A,zhisheng yan is assistant professor in the de...,N/A,"phd, computer science and engineering, state u...",https://cec.gmu.edu/node/6941,zyan4@gmu.edu,https://mason.gmu.edu/~zyan4/index.html\n
239,240.0,ziwei zhu,assistant professor,N/A,ziwei zhu is broadly interested in data minin...,N/A,"phd, computer science, texas a&m university\n",https://cec.gmu.edu/profiles/zzhu20,zzhu20@gmu.edu,N/A
240,228.0,ziyu yao,assistant professor\ncomputer science\ncollege...,N/A,ziyu yao works on natural language processing...,N/A,"phd, computer science and engineering, ohio st...",https://cec.gmu.edu/node/6866,ziyuyao@gmu.edu,N/A


In [ ]:
df2 = df2.replace('ã zlem uzuner', 'ozlem uzuner', regex=True)
df2 = df2.replace('ozlem uzuner', 'özlem uzuner', regex=True)

In [258]:
#Save file
df2.to_csv('Final_Data_For_NLP.csv')